In [1]:
import requests
from datetime import date, datetime, timedelta
import pandas as pd
from io import StringIO
import sys
sys.path.append('../')
from database import connect_to_db_upload
import numpy as np
import pytz

In [2]:
# Function to format CVS
def format_csv(csv_path):
    # CSV file into a dataframe and format to have datetime and numeric columns
    df = pd.read_csv(csv_path, names=['date','counts'])[1:] # Remove extra first row
    df["date"] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.tz_convert('UTC') # convert time zone to UTC
    df = df.set_index('date')
    df["counts"] = df["counts"].apply(pd.to_numeric)
    return df

In [3]:
def fetch_weather(my_station, enddt, startdt):
    """Main loop."""
    # print('Entered fetch_weather function')
    # Step 1: Fetch global METAR geojson metadata
    # https://mesonet.agron.iastate.edu/sites/networks.php
    req = requests.get(
        "http://mesonet.agron.iastate.edu/geojson/network/AZOS.geojson",
        timeout=60,
    )
    geojson = req.json()
    for feature in geojson["features"]:
        station_id = feature["id"]
        if station_id == my_station:
            
            props = feature["properties"]
            # We want stations with data to today (archive_end is null)
            if props["archive_end"] is None:
                print('archive_end is null = data to today')

            # print(f'Fetching data for station {station_id}')
            # uri = (
            #     "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
            #     f"station={station_id}&data=all&year1=1928&month1=1&day1=1&"
            #     f"year2={enddt.year}&month2={enddt.month}&day2={enddt.day}&"
            #     "tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&"
            #     "direct=yes&report_type=3"
            # )
            uri = (
                "http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?"
                f"station={station_id}&data=all&year1={startdt.year}"
                f"&month1={startdt.month}&day1={startdt.day}&"
                f"year2={enddt.year}&month2={enddt.month}&day2={enddt.day}&"
                "tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&"
                "direct=yes&report_type=3"
            )
            # print('uri: ', uri)

            res = requests.get(uri, timeout=300)
            # print('received response type: ', type(res))
            return res

In [4]:
def download_and_save_weather_table():

    # Home directory
    homedir = '../data/'
    # Get station ids from detector settings
    monitors = pd.read_csv('../detector_info_settings/detector_locations.csv')
    ids = monitors['weather_station'].to_list()
    ids2 = list(set(monitors['weather_station'].to_list()))
    print('ids: ', ids)
    
    for my_station in ids2:
        print('Station id: ', my_station, ' counts: ', ids.count(my_station))
        # If several detectors use same station, find the earliest date available
        if ids.count(my_station) > 1:
            names = monitors.loc[monitors['weather_station'] == my_station, 'name'].to_list()
            oldest_dates = []
            for detector_name in names:
                # If single level folder
                try:
                    detector_csv = f'{homedir}{detector_name}/{detector_name}_all_logs.csv'
                    df = pd.read_csv(detector_csv)
                    if 'Unnamed: 0' in df.columns.to_list():
                        df = df.rename(columns={'Unnamed: 0':'date'})
                    df['date'] = pd.to_datetime(df['date'])
                    df = df.set_index('date')

                    # get start and end dates of df
                    df.sort_index(inplace=True)
                    oldest_dates.append(df.head(1).index.values[0])

                # Double level folder like Colombo and Serbia
                except:
                    if 'Det' in detector_name:
                        subfolder = detector_name[-4:]
                        folder = detector_name[:-5]
                    else:
                        subfolder = detector_name[-2:]
                        folder = detector_name[:-3]
                    
                    detector_csv = f'{homedir}{folder}/{subfolder}/{detector_name}_all_logs.csv'
                    df = pd.read_csv(detector_csv)
                    if 'Unnamed: 0' in df.columns.to_list():
                        df = df.rename(columns={'Unnamed: 0':'date'})
                    df['date'] = pd.to_datetime(df['date'])
                    df = df.set_index('date')

                    # get start and end dates of df
                    df.sort_index(inplace=True)
                    oldest_dates.append(df.head(1).index.values[0])
            
            oldest_df = pd.DataFrame(oldest_dates, columns=['date'])
            oldest_df.sort_values(by=['date'], inplace=True)
            oldest_ts = oldest_df.head(1)['date'].item()
            print('oldest date: ', oldest_ts)


        # station for single monitor
        else:
            detector_name = monitors.loc[monitors['weather_station'] == my_station, 'name'].item()
            detector_csv = f'{homedir}{detector_name}/{detector_name}_all_logs.csv'
            df = pd.read_csv(detector_csv)
            if 'Unnamed: 0' in df.columns.to_list():
                df = df.rename(columns={'Unnamed: 0':'date'})
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date')

            # get start and end dates of df
            df.sort_index(inplace=True)
            oldest = df.head(1).index.values[0]
            oldest_ts = datetime.fromtimestamp(
                ((oldest - np.datetime64('1970-01-01T00:00:00'))/ np.timedelta64(1, 's')), 
                tz=pytz.utc)

        # fetch
        weatherjson = fetch_weather(my_station, date.today(), oldest_ts)
        # Read as cvs from json file format
        wdf = pd.read_csv(StringIO(weatherjson.text), sep=',')
        wdf[wdf=='M'] = np.nan

        # Slice only for needed information based on dates and consider if temperature in farenheit
        # print('Columns: ', wdf.columns.to_list())
        if 'tmpc' in wdf.columns.to_list() and 'tmpf' not in wdf.columns.to_list():
            wdf['tmpc'] = wdf['tmpc'].apply(pd.to_numeric)
            wdf['tmpf'] = (wdf['tmpc'] * 9/5) + 32
        wdf['tmpf'] = wdf['tmpf'].apply(pd.to_numeric)

        if 'mslp' in wdf.columns.to_list():
            wdf['mslp'] = wdf['mslp'].apply(pd.to_numeric)
        else:
            wdf['mslp'] = np.nan
        
        if 'alti' in wdf.columns.to_list():
            wdf['alti'] = wdf['alti'].apply(pd.to_numeric)
        else:
            wdf['alti'] = np.nan
        
        # Rename columns
        wdf = wdf.rename(columns={'valid':'date', 'tmpf':'temp_in_f', 'mslp':'sea_l_pressure_millibar', 'alti':'alti_pressure'})

        # Transform dates and make into index
        wdf['date'] = pd.to_datetime(wdf['date'])
        wdf = wdf[['date','temp_in_f', 'sea_l_pressure_millibar', 'alti_pressure']]
        wdf = wdf.set_index('date')
        wdf.sort_index(inplace=True, ascending=True)

        # Resample as an hourly df
        wdf = wdf.resample('h').mean()

        # Remove any 0 values and make into np.nan
        wdf.loc[wdf['temp_in_f'] == 0] = np.nan
        wdf.loc[wdf['sea_l_pressure_millibar'] == 0] = np.nan
        wdf.loc[wdf['alti_pressure'] == 0] = np.nan

        # Connect to DB via postgresql and send to db
        engine, conn = connect_to_db_upload()
        wdf.to_sql(
            con=engine, name=f'{my_station.lower()}', if_exists='replace', index_label='date')
        print(f'Table {my_station.lower()} sent to DB successfully')

        # Make primary key for table via PSYCOPG2
        cur = conn.cursor()
        cur.execute(f"""ALTER TABLE {my_station.lower()} ADD PRIMARY KEY (date)""")
        conn.commit()
        cur.close()
        print('Query for primary key sent successfully')


In [8]:
download_and_save_weather_table()

ids:  ['CQT', 'SKSM', 'DNAA', 'ALM', 'LYBE', 'LYBE', 'VCBI', 'VCBI', 'ATL', 'ATL', 'ATL', 'ATL', 'ATL', 'VCRI']
Station id:  VCBI  counts:  2
oldest date:  2023-09-15 22:00:00
archive_end is null = data to today
Connected to muon database successfully
Connected to muon database successfully
Table vcbi sent to DB successfully
Query for primary key sent successfully
